# New Discussion Tool Instrumentation QA

1. [Differentiate between events emitted from the Reply Tool and the New Discussion Tool](#Differentiate-between-events-emitted-from-the-Reply-Tool-and-the-New-Discussion-Tool)
2. [Differentiate between edits to existing sections and the creation of new sections](#Differentiate-between-edits-to-existing-sections-and-the-creation-of-new-sections)

## Differentiate between events emitted from the Reply Tool and the New Discussion Tool

[Task](https://phabricator.wikimedia.org/T265099)

The EditAttemptStep schema's existing init_type field will be used to differentiate between events emitted from the Reply Tool and the New Discussion Tool.

Events from the Reply Tool and New Discussion Tool should be logged as follows:

* Reply Tool events: event.action = 'init', event.integration = 'discussiontools', event.init_type = 'page'
* New Discussion Tool events: event.action = 'init', event.integration = 'discussiontools', event.init_type = 'section'

The change to the the `init_type` field was made on 12 January 2021.

In [9]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(magrittr); library(zeallot); library(glue); library(tidyverse); library(zoo); library(lubridate)
    library(scales)
})

In [87]:
# Collect init events by discussion tool type
query <-
"
SELECT 
  CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) as date,
  wiki AS wiki,
  event.editing_session_id AS session_id,
  event.platform as platform,
  event.editor_interface as interface,
  event.init_mechanism as init_mechanism,
  IF(event.init_type = 'section', 'new discussion tool', 'reply tool') as dt_type,
  COUNT(*) as n_events
FROM event.editattemptstep
WHERE
  event.action = 'init'
  AND event.integration = 'discussiontools'
  AND year = 2021
  AND dt >= '2021-01-01'
GROUP BY
  CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')),
  wiki, 
  event.editing_session_id,
  event.init_mechanism,
  event.platform,
  event.editor_interface,
  IF(event.init_type = 'section', 'new discussion tool', 'reply tool') 
"

In [88]:
collect_init_events <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



## Reply Tool vs New Discussion Tool Events by Date 

In [89]:
dt_events_bytype <- collect_init_events %>%
    group_by(date, dt_type) %>%
    summarise(total_events = sum(n_events))

dt_events_bytype

`summarise()` regrouping output by 'date' (override with `.groups` argument)



date,dt_type,total_events
<chr>,<chr>,<int>
2021-01-01,reply tool,80
2021-01-02,reply tool,83
2021-01-03,reply tool,91
2021-01-04,reply tool,78
2021-01-05,reply tool,89
2021-01-06,reply tool,76
2021-01-07,reply tool,68
2021-01-08,reply tool,64
2021-01-09,reply tool,59


Both reply and new discussion tool events are being logged and it possible to differentiate based on the `init_type`. There are fewer `init_type = section` events as these are associated with the new discussion tool, which has not been deployed as long as the reply tool events. 

A total of 24 new discussion tool events have been logged since 21 January 2021 as expected.


## Reply Tool vs New Discussion Tool Events by Platform and Editor Interface

In [90]:
dt_events_byplatform <- collect_init_events %>%
    group_by(dt_type, platform, interface) %>%
    summarise(total_events = sum(n_events))

dt_events_byplatform

`summarise()` regrouping output by 'dt_type', 'platform' (override with `.groups` argument)



dt_type,platform,interface,total_events
<chr>,<chr>,<chr>,<int>
new discussion tool,desktop,visualeditor,8
new discussion tool,desktop,wikitext,16
reply tool,desktop,visualeditor,827
reply tool,desktop,wikitext,1227


Events are recorded for both visualeditor and wikitext and only on platform as expected.

##  New Discussion Tool Events and Unique Sessions by Wiki

In [91]:
dt_events_bywiki <- collect_init_events %>%
    filter(dt_type == "new discussion tool") %>%
    group_by(dt_type, wiki) %>%
    summarise(total_events = sum(n_events),
             distinct_sessions = n_distinct(session_id))

dt_events_bywiki

`summarise()` regrouping output by 'dt_type' (override with `.groups` argument)



dt_type,wiki,total_events,distinct_sessions
<chr>,<chr>,<int>,<int>
new discussion tool,cswiki,7,7
new discussion tool,cswikinews,1,1
new discussion tool,enwiki,16,16


In [ ]:
New discussion tool events have been recorded on enwiki, cswikinews, and cswiki.

## Reply Tool vs New Discussion Tool Events by Init Mechanism

In [54]:
dt_events_bymechanism <- collect_init_events %>%
    group_by(dt_type, init_mechanism) %>%
    summarise(total_events = sum(n_events))

dt_events_bymechanism 

`summarise()` regrouping output by 'dt_type' (override with `.groups` argument)



dt_type,init_mechanism,total_events
<chr>,<chr>,<int>
new discussion tool,click,24
reply tool,click,2048


Both the new discussion tool and reply tool events to date have been recorded as click events; however, `Init_mechanism` is not not needed to distinguish these two event types so this is fine. Changes will be needed to track new section events using the existing workflow, which will be done as part of [T272544](https://phabricator.wikimedia.org/T272544)

## Reply Tool vs New Discussion Tool Edit Completion Rate

Check to make sure it will be possible to calculate edit completion rate for each tool type, which is one of the key metrics for this tool.

In [71]:
query <- 
"WITH init_sessions AS (
--first find all dt and reply tool events based on init type
SELECT 
  event.editing_session_id AS session_id,
  IF(event.init_type = 'section', 'new discussion tool', 'reply tool') as dt_type,
  wiki AS wiki
FROM event.editattemptstep
WHERE
  year = 2021 
  AND dt >= '2021-01-12'  -- when instrumetation was deployed
  AND event.action = 'init'
  AND event.integration= 'discussiontools'
)

-- Find associated savesuccess events
SELECT
  eas.event.user_editcount AS edit_count,
  eas.event.user_id AS user,
  init_sessions.dt_type as dt_type,
  eas.event.editing_session_id AS session_id,
  eas.wiki AS wiki,
  COUNT(*) AS save_events
FROM event.editattemptstep eas
INNER JOIN
    init_sessions 
    ON eas.event.editing_session_id = init_sessions.session_id 
    AND eas.wiki = init_sessions.wiki
WHERE
  year = 2021 
-- events since deployment date
  AND dt >= '2021-01-12'
  AND eas.event.action = 'saveSuccess'
  AND eas.event.integration= 'discussiontools'
-- remove anonymous users
  AND eas.event.user_id != 0
GROUP BY 
  eas.event.user_id,
  init_sessions.dt_type,
  eas.event.user_editcount,
  eas.event.editing_session_id,
  eas.wiki
"

In [72]:
collect_savesuccess_events <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [99]:
dt_save_events_bytype <- collect_savesuccess_events %>%
    group_by ( dt_type)  %>%
    summarize (num_save_sessions = n_distinct(session_id),
              num_save_events = sum(save_events))

dt_save_events_bytype

`summarise()` ungrouping output (override with `.groups` argument)



dt_type,num_save_sessions,num_save_events
<chr>,<int>,<int>
new discussion tool,6,6
reply tool,856,856


In [100]:
new_dt_save_events_bywiki <- collect_savesuccess_events %>%
    filter(dt_type == 'new discussion tool') %>%
    group_by (wiki, dt_type)  %>%
    summarize (num_save_sessions = n_distinct(session_id))

new_dt_save_events_bywiki

`summarise()` regrouping output by 'wiki' (override with `.groups` argument)



wiki,dt_type,num_save_sessions
<chr>,<chr>,<int>
cswiki,new discussion tool,1
enwiki,new discussion tool,5


A total of 6 new discussion tool sessions met `saveSuccess`. These are the same wikis where new discussion init events were also logged.


# Differentiate between edits to existing sections and the creation of new sections

[Task](https://phabricator.wikimedia.org/T272544)

## Background

New events were added to EditAttemptStep to enable the software to distinguish edits to existing sections from edits associated with the creation of new sections.

Notes re instrumentation:
- This is explicitly a change to the existing logging in VisualEditor / WikiEditor to get the semantics of init_mechanism tweaked so that (a) they're consistent, and (b) you can always tell whether there's a new section being created.
- No impact or change to DiscussionTools instrumentation, unless and until we implement a takeover for the section=new URL.
- These patches add a new possible value for `init_mechanism: 'url-new'`. It will be logged when direct navigation occurs to a URL that triggers an editor for either a new page or a new section.
- The VE patch also makes its use of `init_mechanism=new` consistent with WikiEditor/the-schema-docs. It was previously not using new when you clicked the "Create" tab after navigating to a non-existent page. (You could still distinguish this case by looking for page_id=0.)





In [5]:
# Collect all init events by date since deployment
query <-
"
SELECT 
  CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) as edit_dt,
  wiki AS wiki,
  event.editing_session_id AS session_id,
  event.editor_interface as interface,
  event.init_mechanism as init_mechanism,
  event.init_type as init_type,
  event.integration as integration,
  COUNT(*) as n_events
FROM event.editattemptstep
WHERE
  event.action = 'init'
  AND year = 2021
-- review events following deployment
  AND dt >= '2021-02-15'
  AND event.platform = 'desktop'
GROUP BY
  CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')),
  wiki, 
  event.editing_session_id,
  event.init_mechanism,
  event.editor_interface,
  event.init_mechanism,
  event.init_type,
  event.integration
"

In [6]:
collect_init_events_all <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



## Init Events by Init Mechanism and Type


In [11]:
init_events_bymechanism <- collect_init_events_all %>%
    group_by(init_mechanism, init_type) %>%
    summarise(n_events = sum(n_events))

init_events_bymechanism

`summarise()` regrouping output by 'init_mechanism' (override with `.groups` argument)



init_mechanism,init_type,n_events
<chr>,<chr>,<int>
click,page,244735
click,section,181952
new,page,156614
new,section,7989
url,page,112989
url,section,165317
url-new,page,223159
url-new,section,2285


We are logging both `init_mechanism = 'url-new'` and `init_mechanism = 'url` events for page and section init_types. This will allow us to now distinguish between new and existing section or page edits that occur from direct naviagation to a URL from existing logging in VisualEditor/WikiEditor. 

The number of logged events for each init_mechanism type seem reasonable given the liklihood of occurrence. The majority of new section edits (78%) are created by clicking on a link to a page vs direct naviagation to a URL that triggers an editor for a new section. The majority of edits from direct navigation to a url (98.9%) are to `init_type = page` events.


## Init Events By Integration Type

In [12]:
init_events_byintegration <- collect_init_events_all %>%
    group_by(init_mechanism, init_type, integration) %>%
    summarise(n_events = sum(n_events))

init_events_byintegration

`summarise()` regrouping output by 'init_mechanism', 'init_type' (override with `.groups` argument)



init_mechanism,init_type,integration,n_events
<chr>,<chr>,<chr>,<int>
click,page,discussiontools,9418
click,page,page,235317
click,section,discussiontools,459
click,section,page,181493
new,page,page,156614
new,section,page,7989
url,page,page,112989
url,section,page,165317
url-new,page,page,223159


Discussion tool events (as indicated by `event.integration = 'discussiontools`) are only recorded for `init_mechnaism = click` events (No `event.mechanism = new` events were recorded for these events). THis is expected as per DLynch's comment, the changes were not implemented for DiscussionTools. We can still distinguish new vs existing events for discussion tools based on current instrumentation. See https://phabricator.wikimedia.org/T265099 for details. 

Non-Discussion tool events (as indicated by `event.integration = 'page`) are recorded for all expected init_mechamism types (click, new, url, and url-new) for both page and section types.

## Init Events By Editor Interface

In [13]:
init_events_byinterface <- collect_init_events_all %>%
    group_by(init_mechanism, init_type, interface) %>%
    summarise(n_events = sum(n_events))

init_events_byinterface

`summarise()` regrouping output by 'init_mechanism', 'init_type' (override with `.groups` argument)



init_mechanism,init_type,interface,n_events
<chr>,<chr>,<chr>,<int>
click,page,visualeditor,37021
click,page,wikitext,204263
click,page,wikitext-2017,3451
click,section,visualeditor,20846
click,section,wikitext,159063
click,section,wikitext-2017,2043
new,page,visualeditor,3393
new,page,wikitext,153200
new,page,wikitext-2017,21


All init_mecahnism types are recorded for all three editor interfaces as expected. The `init_mechanism = new` is now recorded for VisualEditor events. 

## Init Events on Talk Pages Only

In [19]:
# Collect all init events on talk pages only by date since deployment
query <-
"
SELECT 
  CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) as edit_dt,
  wiki AS wiki,
  event.editing_session_id AS session_id,
  event.editor_interface as interface,
  event.init_mechanism as init_mechanism,
  event.init_type as init_type,
  event.integration as integration,
  COUNT(*) as n_events
FROM event.editattemptstep
WHERE
  event.action = 'init'
  AND year = 2021
-- review events following deployment
  AND dt >= '2021-02-15'
  AND event.platform = 'desktop'
 -- review all talk namespaces
  AND event.page_ns % 2 = 1
GROUP BY
  CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')),
  wiki, 
  event.editing_session_id,
  event.editor_interface,
  event.init_mechanism,
  event.editor_interface,
  event.init_mechanism,
  event.init_type,
  event.integration
"

In [20]:
collect_init_events_talkonly <- wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [21]:
head(collect_init_events_talkonly)

,edit_dt,wiki,session_id,interface,init_mechanism,init_type,integration,n_events
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,2021-02-15,abwiki,92c42880482c2ee6e2a5afdce7aee3e4,wikitext,url,page,page,1
2,2021-02-15,acewiki,10f0051052705382eaa8b72e6fb8848f,wikitext,url,page,page,1
3,2021-02-15,afwiki,293c83b05e3e35449759039616f67fb3,wikitext,click,page,page,1
4,2021-02-15,afwiktionary,3561b9c0202922f1a30ab028fa056756,wikitext,url-new,page,page,1
5,2021-02-15,arwiki,1cb977f06ad501ba75cd26c1f6f7b6be,wikitext,new,page,page,1
6,2021-02-15,arwiki,9648bf608a5058d0f2bfb8c0dda02692,wikitext,new,page,page,1


## Talk Page Init Events by Integration and Init Type

In [22]:
talk_init_events_byintegration <- collect_init_events_talkonly %>%
    group_by(init_mechanism, init_type, integration) %>%
    summarise(n_events = sum(n_events))

talk_init_events_byintegration

`summarise()` regrouping output by 'init_mechanism', 'init_type' (override with `.groups` argument)



init_mechanism,init_type,integration,n_events
<chr>,<chr>,<chr>,<int>
click,page,discussiontools,6452
click,page,page,7242
click,section,discussiontools,443
click,section,page,7319
new,page,page,25091
new,section,page,6097
url,page,page,4387
url,section,page,3226
url-new,page,page,20379


All expected init types are logged on talk pages. The number of logged events for each init_mechanism type seem reasonable given the liklihood of occurrence. 

The majority of new section edits on talk pages (80%) are created by clicking on a link to a page vs direct naviagation to a URL that triggers an editor for a new section. The majority of edits from direct navigation to a url (92.9%) are to `init_type = page` events.



## Talk Page Init Events by Editor Interface

In [25]:
talk_init_events_byinterface <- collect_init_events_talkonly %>%
    group_by(integration, init_mechanism, init_type, interface) %>%
    summarise(n_events = sum(n_events))

talk_init_events_byinterface

`summarise()` regrouping output by 'integration', 'init_mechanism', 'init_type' (override with `.groups` argument)



integration,init_mechanism,init_type,interface,n_events
<chr>,<chr>,<chr>,<chr>,<int>
discussiontools,click,page,visualeditor,2841
discussiontools,click,page,wikitext,3611
discussiontools,click,section,visualeditor,141
discussiontools,click,section,wikitext,302
page,click,page,visualeditor,2
page,click,page,wikitext,7123
page,click,page,wikitext-2017,117
page,click,section,visualeditor,1
page,click,section,wikitext,7214


Most all visual editor events on talk pages are recorded for discussiontools events as expected. 

## Summary of New and Existing Section Edits on Talk Pages

In [36]:
existing_vs_new_section_edits <- collect_init_events_talkonly %>%
    filter(integration == 'page',
           init_type == 'section') %>%
    mutate(section_status = ifelse(init_mechanism == 'url-new'|init_mechanism == 'new', 'new', 'existing'))%>%
    group_by(section_status, init_mechanism) %>%
    summarise(n_events = sum(n_events))

existing_vs_new_section_edits

`summarise()` regrouping output by 'section_status' (override with `.groups` argument)



section_status,init_mechanism,n_events
<chr>,<chr>,<int>
existing,click,7319
existing,url,3226
new,new,6097
new,url-new,1543


The table above shows the number of new vs existing section edits using current VisualEditor / WikiEditor section workflows (not disucssion tool related edits).

The number of logged events for each section edit type seem reasonable given the liklihood of occurrence. Since deployment of the instrumentation changes, a slight majority of section edits on talk pages have been to existing sections (58%). 

Most edits (79.8%) to create new sections on talk pages are completed by clicking the link on the page vs direct navigation to a url as expected.
